# Crowler

In [22]:
!pip install google-play-scraper

In [23]:
app_name= 'br.com.zeroum.turmadamonica' #id do app na play store

In [24]:
from google_play_scraper import Sort, reviews_all
import pandas as pd

result = reviews_all(
  app_name,
  sleep_milliseconds=0,
  lang='en', 
  sort=Sort.NEWEST,
)

df_app_review=pd.DataFrame(result)
df_app_review

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,gp:AOqpTOFyRa5DW9IITumBRG7kPCi8q-m7RtDx9rWwrSx...,Nalbert To play!!!,https://play-lh.googleusercontent.com/a-/AOh14...,This game is very good but not to put the epis...,5,2,2.5.3,2021-05-23 17:44:33,None,None
1,gp:AOqpTOH0D9t-byAia9kuLiG4IPmZhiojVICe3cGvM6K...,Josh verduzco,https://play-lh.googleusercontent.com/a/AATXAJ...,☺☺☺☺☺☺☺☺qoop qop. qop?,5,2,None,2021-03-30 23:11:09,None,None
2,gp:AOqpTOFv-ilsMiUsWTPvNeiolpWhA-rYUg_fnIJjY4N...,Silva Kabamba,https://play-lh.googleusercontent.com/a-/AOh14...,A Mei turma da Mônica 😊😊😊😊😊😘,5,10,2.5,2020-05-23 23:51:39,None,None
3,gp:AOqpTOGa1jHHreayvDX46GS-PmCy7CeZz3kCf6F5s62...,anca lupu,https://play-lh.googleusercontent.com/a/AATXAJ...,Monica toy ❤️,5,10,None,2020-02-21 16:08:03,None,None
4,gp:AOqpTOFk3e4uq5G2HnEkmJrY8MBcp9SaB_85r3o1NWz...,Erick Feliciano,https://play-lh.googleusercontent.com/a/AATXAJ...,funny cartoon show.,5,19,1.9.1,2019-10-31 00:14:33,None,None
5,gp:AOqpTOGBcTYuvyQFLEzB2PPCqga23WXgacSa5OAbumT...,Tony Horton,https://play-lh.googleusercontent.com/a-/AOh14...,😍😍😍😍😍😍😍😍😍😍😍😍😍😍😍🍦😍😍😍😍😍😍😍😍😍😍😍😍😍😑😍😍😍😍,5,11,None,2019-06-01 22:34:34,None,None
6,gp:AOqpTOGRZBbgvf6jzgHc0BlzAt0mfhiAQzwPeJjoWQE...,Luciana Pontes,https://play-lh.googleusercontent.com/a-/AOh14...,Bomdiapraquemgostadese,5,15,None,2018-08-25 19:45:45,None,None


# Load Model

## At BFRU Dataset

In [25]:
!wget https://raw.githubusercontent.com/adailtonaraujo/app_review_analysis/master/Classification/Models/MNB_RoBERTa_BFRU.pkl

--2021-08-05 22:19:11--  https://raw.githubusercontent.com/adailtonaraujo/app_review_analysis/master/Classification/Models/MNB_RoBERTa_BFRU.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66438 (65K) [application/octet-stream]
Saving to: ‘MNB_RoBERTa_BFRU.pkl.1’

MNB_RoBERTa_BFRU.pk 100%[===================>]  64.88K  --.-KB/s    in 0.005s  

2021-08-05 22:19:11 (13.3 MB/s) - ‘MNB_RoBERTa_BFRU.pkl.1’ saved [66438/66438]



In [26]:
pkl_filename1 = "MNB_RoBERTa_BFRU.pkl"

## At ARE Dataset


In [27]:
!wget https://raw.githubusercontent.com/adailtonaraujo/app_review_analysis/master/Classification/Models/SVM_RoBERTa_ARE.pkl

--2021-08-05 22:19:11--  https://raw.githubusercontent.com/adailtonaraujo/app_review_analysis/master/Classification/Models/SVM_RoBERTa_ARE.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19242104 (18M) [application/octet-stream]
Saving to: ‘SVM_RoBERTa_ARE.pkl.1’

SVM_RoBERTa_ARE.pkl 100%[===================>]  18.35M  --.-KB/s    in 0.1s    

2021-08-05 22:19:11 (175 MB/s) - ‘SVM_RoBERTa_ARE.pkl.1’ saved [19242104/19242104]



In [28]:
pkl_filename2 = "SVM_RoBERTa_ARE.pkl"

## LOAD


In [29]:
pkl_filename = pkl_filename2

In [30]:
import pickle
with open(pkl_filename, 'rb') as file:
  clf = pickle.load(file)

# Represent User Text (USE GPU)



In [31]:
!pip install -U sentence-transformers

In [32]:
from sentence_transformers import SentenceTransformer
import numpy as np

In [33]:
import pandas as pd
def WordEmbeddings(texts, algorithm):

  model = SentenceTransformer('roberta-large-nli-stsb-mean-tokens')

  if type(texts) == pd.core.series.Series:
    sentences = texts.replace(['\t','\n','\r'], [' ',' ',' '], regex=True)
  else:
    sentences = texts

  sentence_embeddings = model.encode(list(sentences))

  if algorithm == 'MNB':
    return np.abs(np.min(sentence_embeddings)) + sentence_embeddings
  elif algorithm == 'SVM':
    return sentence_embeddings

The variable *txts* can be a dataframe column or a list of strings




# DEMO

In [34]:
data = WordEmbeddings(df_app_review.content,pkl_filename.split('_')[0])

predicted_classes = clf.predict(data)
predicted_classes

array(['irr', 'irr', 'irr', 'irr', 'irr', 'irr', 'irr'], dtype='<U3')

# Classification Metrics


**y_test_class on BFRU:**


*   Bug
*   Feature
*   User Experience
*   Rating

**y_test_class on ARE:**

*   pbr
*   iqr
*   irr






In [35]:
y_test_class = ['irr', 'irr', 'irr', 'irr', 'irr', 'irr', 'irr']

In [36]:
from sklearn.metrics import classification_report

print(classification_report(y_test_class, predicted_classes, output_dict=False))

              precision    recall  f1-score   support

         irr       1.00      1.00      1.00         7

    accuracy                           1.00         7
   macro avg       1.00      1.00      1.00         7
weighted avg       1.00      1.00      1.00         7

